In [7]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
stops = set(stopwords.words("english")) | set(stopwords.words("russian"))
from sklearn.metrics import mean_squared_error, accuracy_score
import re
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
    100% |████████████████████████████████| 51kB 2.5MB/s 
    100% |████████████████████████████████| 7.1MB 5.0MB/s 


In [8]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving train.csv to train.csv
User uploaded file "train.csv" with length 149556868 bytes


In [11]:
df = pd.read_csv('train.csv')
def clean(text):
    
    text = re.sub("[^а-яА-Яa-zA-Z]"," ", text)
    words = text.lower().split()
    words = [w for w in words if not w in stops]
    words = [morph.parse(w)[0].normal_form for w in words ]
    return(words)
  
for i in tqdm(range(len(df))):
    df.iloc[i]['text'] = clean(df.iloc[i]['text'])

df.sample(frac = 1, random_state = 42).head()
df.to_csv('clean.csv', sep=';', encoding='utf-8')

100%|██████████| 3485/3485 [27:41<00:00,  1.20it/s]


In [0]:

clean_df = pd.read_csv("clean.csv", sep=";")

In [0]:
def clean2(string):
    string = ''.join(c for c in string)
    spliced = string.split(' ')
    string = ''.join(word + ' ' for word in spliced if len(word) > 3 )
    return string[:-1]

for i in range(len(df)):
    df.iloc[i]['text'] = clean2(df.iloc[i]['text'])
    
df.to_csv('clean2.csv', sep=';', encoding='utf-8')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [0]:
df = pd.read_csv('clean2.csv', sep = ';')

In [55]:
df['text'][0]

'страницабумажныйверсияномеригорьпавлоравтормногочисленныйстатькнигагуманитарныйпрофильосновнойнаучныйинтерессосредоточитьтеорияисторияжитьконстанцагерманиясанктпетербургcкоротатьраспадбориспастернакстрахвырождениеодинконстантамирвыстраиватьчеловеккраеугольныйпунктсоциокультураневерныйусматриватьпервопричинастрахфизиологиядряхлениеиндивидуальныйорганизмоновосполнитьпродолжениеродстарениедолжныйбеспокоитьлишьбиологическийособьоднакозаботитьсясохранениежизньпотомствоодинмоментпоявлениеземляhomosapiensпечьсяумершийухаживатьмогилаучреждатькультпредоксвойотчуждённостьприродачеловекжаждатьпопастьсугубомыслитьреальностьтудаотсутствоватьфизическипребываниедвояконеадекватныйбытьсоположитьприходитьсменактосветпоэтомувсякийновыйначаловидетьсяконецрождениесмертькоторыйпартиципироватьподдерживатьсвязьуйтипоколениеэросчеловекпримешиватьтанатосдавноуказатьсабинашпильрейнстатьядеструкцияпричинастановлениежоржбатайпредисловиемадамэдвардпроцесснаследованиеобратитьбудущеепрошлоевкрадыватьсяфундаментальны

In [0]:
y = [int(string[9:]) for string in df.author.values] # remove "__label__" from string and convert to int

xtrain, xvalid, ytrain, yvalid = train_test_split(train_df.text.values, y, 
                                                  random_state=42, 
                                                  test_size=0.1,)

In [0]:
tfv = TfidfVectorizer(min_df=3, analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1) 

tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain)
xvalid_tfv = tfv.transform(xvalid)

In [22]:
regr_1 = LogisticRegression(C=1000, penalty = 'l2' )
regr_1.fit(xtrain_tfv, ytrain)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


NameError: ignored

In [23]:
predictions_1 = regr_1.predict(xvalid_tfv)
accuracy_score(predictions_1, yvalid)

0.9054441260744985

Пробуем word2vec


In [25]:
from gensim.models import word2vec
model = word2vec.Word2Vec(df['text'], size=300, window=10, workers=4)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
from collections import defaultdict
class mean_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
class tfidf_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [0]:
xtrain_mean = mean_vectorizer(w2v).fit(xtrain).transform(xtrain)
xvalid_mean = mean_vectorizer(w2v).fit(xvalid).transform(xvalid)


In [0]:
xtrain_w2v_tfv = tfidf_vectorizer(w2v).fit(xtrain).transform(xtrain)
xvalid_w2v_tfv = tfidf_vectorizer(w2v).fit(xvalid).transform(xvalid)

In [28]:
regr_2 = LogisticRegression()
regr_2.fit(xtrain_mean, ytrain)
predictions_2 = regr_2.predict(xvalid_mean)
accuracy_score(predictions_2, yvalid)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.11174785100286533

In [44]:
regr_2.fit(xtrain_w2v_tfv, ytrain)
prediction = regr_2.predict(xvalid_w2v_tfv)
accuracy_score(prediction, yvalid)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.11461318051575932

In [29]:
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_mean, ytrain)
predictions_3 = clf.predict(xvalid_mean)



NameError: ignored

In [31]:
print ("accuracy: %0.3f " % accuracy_score(yvalid, predictions_3))

accuracy: 0.246 


In [33]:
train_df['text'][3]

"['свет', 'упасть', 'тишина', 'родиться', 'живой', 'том', 'москва', 'окончить', 'литературный', 'институт', 'метр', 'горький', 'печататься', 'поэт', 'критик', 'литературный', 'газета', 'независимый', 'газета', 'журнал', 'знамя', 'октябрь', 'юность', 'новый', 'юность', 'интерпоэзия', 'волга', 'зинзивер', 'обрести', 'свет', 'свет', 'упасть', 'тишина', 'двор', 'человек', 'заблудиться', 'путь', 'это', 'песня', 'сыграть', 'быть', 'хороший', 'сол', 'ной', 'дудочка', 'ледяной', 'труба', 'балалайка', 'струнный', 'тишина', 'леденцовый', 'дать', 'закурить', 'путь', 'тьма', 'два', 'беспалый', 'симфония', 'сигарета', 'нужно', 'вс', 'простить', 'никто', 'понять', 'разучиться', 'радоваться', 'ничто', 'обрести', 'свет', 'ii', 'свет', 'путеводный', 'горбун', 'деревянный', 'фонарь', 'свет', 'путеводный', 'зимний', 'дорога', 'падать', 'падать', 'боль', 'ослепительный', 'ларь', 'пусть', 'возродиться', 'немного', 'перека', 'тся', 'воля', 'тся', 'твой', 'электрод', 'вспомниться', 'снежный', 'морось', 'бара

In [38]:
df_1 = pd.read_csv('train.csv')
print (df_1['text'][1])

Мелочи жизни   Проплаченная культура Конец минувшего года в российской культуре ознаменовался событием, на которое в горячке финансового кризиса не обратили должного внимания. Между тем оно очень даже заслуживает того, чтобы задуматься над ним всерьез. В конце прошлого года в России тихо и незаметно издох постмодернизм. “Зачем же так грубо?” — скажут. Однако, говоря “издох”, я вовсе не желаю еще раз оскорбить постмодернизм и постмодернистов, хотя никогда не скрывал и не скрываю враждебности своего отношения к этому явлению. Но в данном случае слово “издох” не содержит в себе никакого эмоционального наполнения. Это всего только цитата из шуточной эпитафии Владимира Соловьева: В лесу болото, в болоте мох. Родился кто-то, потом издох. Постмодернизм в России и был этим “кем-то”, чье рождение и смерть не имели решительно никакого смысла. Однако отсутствие смысла также в своем роде содержательно. Оно хотя бы обозначает такие места, где искателям смысла искать нечего. О русском постмодернизме